Creating Merged Data

In [2]:
import pandas as pd

#reads all the files
cases = pd.read_csv("../../../data/stage_1/covid_confirmed_usafacts.csv")
population_perCounty = pd.read_csv("../../../data/stage_1/covid_county_population_usafacts.csv")
deaths =  pd.read_csv("../../../data/stage_1/covid_deaths_usafacts.csv")

Dropping the rows with a county FIPS of 0

In [3]:
#drops the 0 countyFIPS
cases = cases.drop(cases.index[cases['countyFIPS'] == 0])
cases.countyFIPS=cases.countyFIPS.apply('{:0>5}'.format)

population_perCounty = population_perCounty.drop(population_perCounty.index[population_perCounty['countyFIPS'] == 0])
population_perCounty.countyFIPS=population_perCounty.countyFIPS.apply('{:0>5}'.format)

deaths = deaths.drop(deaths.index[deaths['countyFIPS'] == 0])
deaths.countyFIPS=deaths.countyFIPS.apply('{:0>5}'.format)

Creating a new index for all three tables 

In [4]:
#creates new index as FIPS + STATE for county
cases_id = cases.countyFIPS.astype(str) + cases.State
cases.index = cases_id
cases.index.name = 'UID'

#creates new index as FIPS + STATE for population
population_perCounty_id = population_perCounty.countyFIPS.astype(str) + population_perCounty.State
population_perCounty.index = population_perCounty_id
population_perCounty.index.name = 'UID'


#creates new index as FIPS + STATE for deaths
deaths_id = deaths.countyFIPS.astype(str) + deaths.State
deaths.index = deaths_id
deaths.index.name = 'UID'

Does 3 inner joins to create the final table

In [5]:
#inner join cases and population
merged = pd.merge(cases.add_suffix("_cases"),deaths.add_suffix("_deaths"), on='UID')

finalMerger = pd.merge(merged, population_perCounty, on='UID')

finalMerger.head()

correctMerge = finalMerger.T.drop_duplicates().T

del correctMerge['County Name_cases']
correctMerge.head()


,countyFIPS_cases,State_cases,StateFIPS_cases,2020-01-22_cases,2020-01-24_cases,2020-01-26_cases,2020-01-27_cases,2020-01-29_cases,2020-01-31_cases,2020-02-01_cases,...,2021-08-09_deaths,2021-08-10_deaths,2021-08-11_deaths,2021-08-12_deaths,2021-08-13_deaths,2021-08-14_deaths,2021-08-15_deaths,2021-08-16_deaths,County Name,population
UID,,,,,,,,,,,,,,,,,,,,,
01001AL,01001,AL,1,0,0,0,0,0,0,0,...,114,114,114,114,114,114,114,114,Autauga County,55869
01003AL,01003,AL,1,0,0,0,0,0,0,0,...,330,332,338,342,344,344,344,347,Baldwin County,223234
01005AL,01005,AL,1,0,0,0,0,0,0,0,...,63,63,64,64,65,65,65,65,Barbour County,24686
01007AL,01007,AL,1,0,0,0,0,0,0,0,...,66,66,67,67,67,67,67,67,Bibb County,22394
01009AL,01009,AL,1,0,0,0,0,0,0,0,...,140,140,140,140,140,140,140,141,Blount County,57826


In [6]:
correctMerge.to_csv("merged_covid19.csv")
